In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

In [80]:
import pandas as pd
import re
import pprint

## Connexion a spotify

In [3]:
CRED_PATH_SPOTIFY = "../credentials-spotify.json"

In [4]:
with open(CRED_PATH_SPOTIFY, 'r') as handle:
    data = json.load(handle)

In [5]:
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    **data
))

## Connexion a google

In [6]:
CREDENTIALS_PATH_GOOGLE = "../credentials-sheets.json"

In [7]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET = '1b75J-QTGrujSgF9r0_JPOKkcXAwzFVwpETOAyVBw8ak'

In [8]:
# Load service account credentials.
__credentials = Credentials.from_service_account_file(CREDENTIALS_PATH_GOOGLE, scopes=SCOPES)

# Creates Google Sheets API (v4/latest) service.
service = build('sheets', 'v4', credentials=__credentials)

In [9]:
values = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET, range='Notations').execute()['values']
headers = values.pop(0)

In [10]:
df = pd.DataFrame(values, columns=headers)

# DF preprocessing

In [11]:
df.set_index(['genre', 'sub_genre', 'artist', 'album', 'song'], inplace=True)
df = df.apply(lambda s: s.str.replace(",", "."))
df = df.apply(lambda s: pd.to_numeric(s, errors='coerce'))

In [13]:
df.describe()

,ntQ,ntG,ntV,ntR,ntS,ntGl,ntRx,ntC,ntL
count,1290.000000,569.000000,737.000000,207.000000,251.000000,113.000000,19.000000,20.000000,14.000000
mean,7.712016,8.116169,7.388738,7.684300,6.653586,7.484513,7.973684,6.435000,6.944286
std,1.295435,0.828106,1.343893,1.592582,1.945438,1.724949,2.288095,1.805044,3.057809
min,0.000000,4.000000,0.000000,0.100000,0.050000,0.000000,2.000000,0.000000,0.000000
25%,7.500000,7.750000,7.000000,7.250000,5.000000,7.000000,7.500000,5.950000,6.847500
50%,8.000000,8.250000,7.500000,8.000000,7.000000,7.500000,8.000000,6.625000,8.000000
75%,8.500000,8.750000,8.500000,8.700000,8.000000,8.500000,10.000000,7.375000,8.875000
max,10.000000,9.500000,9.750000,9.700000,10.000000,10.000000,10.000000,8.750000,9.330000


In [63]:
def eq_ignorecase(a, b):
    return re.search(a, b, re.IGNORECASE)    

In [77]:
def filter_search(row, res):
    songs = res['tracks']['items']
    valid = []
    for song in songs:
        s_artist = song["artists"][0]["name"]
        s_album = song["album"]["name"]
        s_title = song["name"]
        if eq_ignorecase(row["artist"], s_artist) and\
           eq_ignorecase(row["album"], s_album) and\
           eq_ignorecase(row["song"], s_title):
            valid.append(song)
    print(row["artist"], row["album"], row["song"])
    print(len(valid) >= 1)

In [83]:
for idx, (_, content) in enumerate(df.index.to_frame().iterrows()):
    search = f"{content.artist} {content.song} {content.album}"
    res = spotify.search(search)
    pprint.pp(res)
    break
    filter_search(content, res)
    continue
    try:
        track = res['tracks']['items'][0]
    except IndexError as e:
        print(f"{search} not in spotify")
        bad_format.append(search)
        continue
    album = track['album']['name']
    name = track['name']
    artist = track['artists'][0]['name']
    id = track['id']
    print(f"{idx:<4}/{len(df)} : {id} {name} {artist} {album}")

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Abba+I+do%2C+I+do%2C+I+do%2C+I+do%2C+I+do+Abba&type=track&offset=0&limit=10',
            'items': [{'album': {'album_type': 'album',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LcJLqbBmaGUft1e9Mm8HV'},
                                              'href': 'https://api.spotify.com/v1/artists/0LcJLqbBmaGUft1e9Mm8HV',
                                              'id': '0LcJLqbBmaGUft1e9Mm8HV',
                                              'name': 'ABBA',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:0LcJLqbBmaGUft1e9Mm8HV'}],
                                 'available_markets': ['CA', 'MX', 'US'],
                                 'external_urls': {'spotify': 'https://open.spotify.com/album/0C36RlW2Fa0C7n1JnWBBMP'},
                                 'href': 'https://api.spotify.

In [84]:
bad_format

["Alkapote, Vald Plus haut - Les marches de l'empeureur Saison 3 / épisode 1 N/A",
 'Auracle Bombs Away Ballet City Slickers',
 'Auracle City of Penetrating Light City Slickers',
 'Auracle Honey City Slickers',
 'Auracle Little City Slickers City Slickers',
 "Auracle Rotary Andy's Raggedy City Slickers",
 'Auracle Sambanana City Slickers',
 'Auracle Tied Shoes City Slickers',
 "B.J. Thomas Rain Drops Keep Falling on My Head Rain Drops Keep Fallin' On My Head",
 'Casiopea Galactic Funk Crosspoint',
 'Casiopea Swear! Mint Jams',
 'Claude François 17 ans Chansons françaises',
 'Claude François Cette année-là Le Vagabond',
 "Début de Soirée Nuit de Folie Jardins d'enfants",
 'Draconian The Empy Stare / September Ashes Turning Season Within',
 'Eric Burdon Colour Of The Sunset When I Was Young',
 'Flamingosis A Groovy Intro A Groovy Thing',
 'Flamingosis A Groovy Thing A Groovy Thing',
 'Flamingosis Breezy A Groovy Thing',
 'Flamingosis Come & Get it A Groovy Thing',
 'Flamingosis Don’t Los